9) Se cuenta con un RDD con información sobre patentamientos de autos con la
siguiente información (patente, marca, modelo, versión, tipo_vehiculo, provincia, fecha),
donde tipo_vehiculo indica si la unidad patentada es auto, pickup, camión o moto.
Se pide generar un programa en pySpark que indique la marca y modelo del auto más
patentado por tipo de vehículo en la provincia de Buenos Aires en el mes de Abril de 2018. #no tengo del 2017 en mi base de datos

In [18]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

aux = sc.textFile('/home/marcos/PycharmProjects/practicaSpark/Ej9.csv')
vehicles = aux.map(lambda x: x.split(","))
vehicles = vehicles.map(lambda x: (x[0], x[1], x[2],
                         pd.to_numeric(x[3], 'coerce'), x[4], x[5], pd.to_datetime(x[6], 'coerce', format='%Y-%m-%d'))).cache()

vehicles.filter(lambda x: (x[6].year == 2018) & (x[6].month == 4) & (x[5] == 'Buenos Aires')) \
        .map(lambda x: ((x[4], x[1] + ' ' + x[2]), 1)).reduceByKey(lambda x,y: x + y) \
        .map(lambda x: (x[0][0], (x[0][1], x[1]))).reduceByKey(lambda x,y: x if (x[1] > y[1]) else y) \
        .map(lambda x: (x[0], x[1][0])).collect()

[('pickup', 'Audi 2'), ('camion', 'Volkswagen 3')]